# Correlation plot combine and compare

#### Note: Use the epi-paint kernel only.

What is this used for? 
- Once the feature extractor is used to extract features, it saves the information as a *.csv file for each cell. 
- Pearson Correlation for this data is then calculated and plotted. 

Workflow
- Define source parent folder location.
- Search for *.csv files and combine them. 
- Calculate Pearson Correlation of dataset.
- Plot the heatmap for all combinations. 

In [ ]:
# Import Dependencies

import os as _os
import os.path as _ospath
import numpy as _np
import pandas as _pd
import matplotlib.pyplot as _plt
from matplotlib.colors import LogNorm
import seaborn as _sns
from scipy.stats import spearmanr

In [ ]:
# Build and/or define output folder
def output_folder_path(folder):
    output_folder = _ospath.join(folder, 'Analysis', 'Correlation')
    if not _ospath.exists(output_folder):
        _os.makedirs(output_folder)
    return output_folder

# Data combiner
def combine_data(folder, file_name):
    files_list = []
    for root, dirs, files in _os.walk(folder):
        for file in files:
            if file == file_name:
                data_file = _ospath.join(root, file)
                files_list.append(data_file)

    if len(files_list) == 0:
        raise FileNotFoundError(f'No files with name {file_name} found in {folder}.')

    files_list = [file for file in files_list if file.endswith('.csv')] # Important in MacOS as MacOS directories have a .DS_Store file that messes things up without this. 
    print(f'Found {len(files_list)} CSV files with name {file_name} in {folder}.')
    for file in files_list:
        print(file)
    data_list = [_pd.read_csv(file, index_col=0) for file in files_list]
    data = _pd.concat(data_list, ignore_index=True)
    data = data.dropna(how='all')
    data = data.fillna(0)
    return data

# Correlation Plot for Each Data Type
def correlationplot(data1, data2):
    if data1.shape != data2.shape:
        raise ValueError("Dataframes must have the same dimensions.")
    num_datasets = data1.shape[1]
    columns = data1.columns[:num_datasets]
    correlation_matrix = _np.ones((num_datasets,num_datasets))

    for i in range(num_datasets):
        for j in range(num_datasets):
            # Extract the two datasets
            data_i = data1.iloc[:, i]
            data_j = data2.iloc[:, j]

            # Find valid indices where both datasets are nonzero
            valid_idx = (data_i != 0) & (data_j != 0) & (~data_i.isna()) & (~data_j.isna())

            # Compute correlation only if enough nonzero values exist
            if valid_idx.sum() > 1:  # At least 2 points needed for correlation
                corr = _np.corrcoef(data_i[valid_idx], data_j[valid_idx])[0, 1] # Computes Pearson's Correlation
                # corr = spearmanr(data_i[valid_idx], data_j[valid_idx])[0] # Computes Spearman's Correlation
            else:
                corr = _np.nan  # Not enough data to compute correlation

            # Store in the matrix (symmetric)
            correlation_matrix[i, j] = corr

    correlation_matrix = _pd.DataFrame(correlation_matrix, index=columns, columns=columns)

    return correlation_matrix

# Plotting the correlation matrix
def plot_correlation_confusion_matrix_two_conditions(correlation_matrix_bottom_left, correlation_matrix_top_right, output_folder_1, output_folder_2, output_file_name):
    assert correlation_matrix_bottom_left.shape == correlation_matrix_top_right.shape, "Matrices must be the same shape"
    assert (correlation_matrix_bottom_left.index == correlation_matrix_bottom_left.columns).all(), "Bottom left must be a square matrix with matching index/columns"
    assert (correlation_matrix_top_right.index == correlation_matrix_top_right.columns).all(), "Top right must be a square matrix with matching index/columns"
    
    combined_matrix = correlation_matrix_bottom_left.copy() * 0
    
    for i in range(len(correlation_matrix_bottom_left)):
        for j in range(i):
            combined_matrix.iloc[i, j] = correlation_matrix_bottom_left.iloc[i, j]

    for i in range(len(correlation_matrix_top_right)):
        for j in range(i + 1, len(correlation_matrix_top_right)):
            combined_matrix.iloc[i, j] = correlation_matrix_top_right.iloc[i, j]

    max_value = _np.max(combined_matrix[combined_matrix != 1].abs())
    cmap = _sns.color_palette('PuOr_r', as_cmap=True)
    
    ax = _sns.heatmap(combined_matrix, annot=False, linewidths=0.5, square=True, cmap=cmap, vmin = (-1 * max_value), vmax = max_value, mask = combined_matrix.isna(), center=0, )
    _plt.savefig(_os.path.join(output_folder_1, output_file_name), format = 'svg', bbox_inches='tight')
    _plt.savefig(_os.path.join(output_folder_2, output_file_name), format = 'svg', bbox_inches='tight')
    _plt.show()

# Correlation Difference Matrix
def plot_difference_matrix(correlation_condition_bottom_left, correlation_condition_top_right, output_folder_1, output_folder_2, output_file_name):
    difference_matrix = correlation_condition_top_right - correlation_condition_bottom_left
    mask = _np.triu(_np.ones_like(difference_matrix, dtype=bool), k=0)
    difference_matrix = difference_matrix.iloc[1:, :-1]
    max_value = _np.max(difference_matrix[difference_matrix != 1].abs())
    _sns.heatmap(difference_matrix, mask=mask[1:, :-1], annot=False, linewidths=0.5, square=True, cmap='PuOr_r', vmin = (-1 * max_value), vmax = max_value, center = 0, )
    _plt.savefig(_os.path.join(output_folder_1, output_file_name), format = 'svg', bbox_inches='tight')
    _plt.savefig(_os.path.join(output_folder_2, output_file_name), format = 'svg', bbox_inches='tight')
    _plt.show()

In [ ]:
# Two condition plotter

folder_condition_bottom_left = '' # Parent Folder Bottom Left
folder_condition_top_right = '' # Parent Folder Top Right
output_folder_1 = output_folder_path(folder_condition_bottom_left)
output_folder_2 = output_folder_path(folder_condition_top_right)
file_name_1 = 'data_1.csv'
file_name_2 = 'data_1.csv'
data_1 = combine_data(folder_condition_bottom_left, file_name_1)
data_2 = combine_data(folder_condition_bottom_left, file_name_2)
data_3 = combine_data(folder_condition_top_right, file_name_1)
data_4 = combine_data(folder_condition_top_right, file_name_2)
correlation_condition_bottom_left = correlationplot(data_1, data_2)
correlation_condition_top_right = correlationplot(data_3, data_4)
plot_correlation_confusion_matrix_two_conditions(correlation_condition_bottom_left, correlation_condition_top_right, output_folder_1, output_folder_2, output_file_name = 'correlation_matrix_two_conditions.svg')
plot_difference_matrix(correlation_condition_bottom_left, correlation_condition_top_right, output_folder_1, output_folder_2, output_file_name = 'difference_matrix_two_conditions.svg')